# Krylov Subspace Approximation XL-BOMD (KSA-XL-BOMD)
### see https://pubs.acs.org/doi/full/10.1021/acs.jctc.0c00264 for details.

In [4]:
import torch
from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.MolecularDynamics import KSA_XL_BOMD

torch.set_default_dtype(torch.float64)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [5]:
%%time
torch.manual_seed(0)

species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)
coordinates = torch.tensor([
                  [
                   [ 0.00,  0.0,  0.0],
                   [ 1.22,  0.0,  0.0],
                   [ 1.82,  0.94, 0.0],
                   [ 1.82, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.20,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [1], # converger used for scf loop
                   }


molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

# Set max_rank for a low-rank approximation. See https://pubs.acs.org/doi/full/10.1021/acs.jctc.0c00264 for details.
# Set err_threshold for low-rank approximation error tollerance. Set err_threshold=0.0 to keep max_rank constant.
# T_el - electronic temperature for thermmal HF.
# Set k for the dissipative electronic force term. See https://aip.scitation.org/doi/full/10.1063/1.3148075 for details.

output = {
'molid': [0,1],
'prefix': 'Outputs/5_KSA_XL-BOMD',
'print_every': 1,
# write a .xyz file
"xyz": 1,
# write an HDF5 file
"h5": {
    "data": 1,      # write T/Ek/Ep, excitations, MO, etc.; 0 disables
    },
}
xl_bomd_params={'k':6, 'max_rank':3, 'err_threshold':0.0, 'T_el':1500}

md = KSA_XL_BOMD(xl_bomd_params=xl_bomd_params, Temp = 400.0,
                   seqm_parameters=seqm_parameters, timestep=0.4, output=output).to(device)
#remove center of mass velocity
_ = md.run(molecule, 10)

Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   398.75   2.061721e-01 -4.755812e+02 -4.753750e+02 ||    424.80   2.196388e-01 -1.091492e+02 -1.089295e+02 || 
     2   377.09   1.949717e-01 -4.755707e+02 -4.753757e+02 ||    466.20   2.410460e-01 -1.091710e+02 -1.089300e+02 || 
     3   340.14   1.758650e-01 -4.755514e+02 -4.753755e+02 ||    517.41   2.675232e-01 -1.091979e+02 -1.089304e+02 || 
     4   293.25   1.516229e-01 -4.755262e+02 -4.753746e+02 ||    568.75   2.940665e-01 -1.092247e+02 -1.089307e+02 || 
     5   241.68   1.249566e-01 -4.754993e+02 -4.753743e+02 ||    609.19   3.149771e-01 -1.092460e+02 -1.089310e+02 || 
     6   191.30   9.891194e-02 -4.754731e+02 -4.753741e+02 ||    628.85   3.251396e-01 -1.092564e+02 -1.089312e+02 || 
     7   148.11   7.657781e-02 -4.754509e+02 -4.753743e+02 ||    621.72   3.214554e-01 -1.092529e+02 -1.089314e+02 || 
     8   116.26   6.011237e-02 -4.754345e+02 -4.753744e+02 ||    587.72   3.038739e-01 -1.092352e+02 -1.089313e+

## Langevin Thermostat

In [6]:
%%time
torch.manual_seed(0)

species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)
coordinates = torch.tensor([
                  [
                   [ 0.00,  0.0,  0.0],
                   [ 1.22,  0.0,  0.0],
                   [ 1.82,  0.94, 0.0],
                   [ 1.82, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.20,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   }


molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

# Set max_rank for a low-rank approximation. See https://pubs.acs.org/doi/full/10.1021/acs.jctc.0c00264 for details.
# Set err_threshold for low-rank approximation error tollerance. Set err_threshold=0.0 to keep max_rank constant.
# T_el - electronic temperature for thermmal HF.
# Set k for the dissipative electronic force term. See https://aip.scitation.org/doi/full/10.1063/1.3148075 for details.

output = {
'molid': [0,1],
'prefix': 'Outputs/5_KSA_XL-BOMD_Langevin',
'print every': 1,
# write a .xyz file
"xyz": 1,
# write an HDF5 file
"h5": {
    "data": 1,      # write T/Ek/Ep, excitations, MO, etc.; 0 disables
    "coordinates": 1,   # write vel/forces/coords; 0 disables
    },
}
xl_bomd_params={'k':6, 'max_rank':3, 'err_threshold':0.0, 'T_el':1500}

md = KSA_XL_BOMD(xl_bomd_params=xl_bomd_params, damp=50.0,
                   seqm_parameters=seqm_parameters, Temp=400.0, timestep=0.4, output=output).to(device)
#remove center of mass velocity
_ = md.run(molecule, 10, remove_com=('angular',1))

Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   400.71   2.071846e-01 -4.755815e+02 -4.753743e+02 ||    436.12   2.254929e-01 -1.091469e+02 -1.089214e+02 || 
     2   388.46   2.008505e-01 -4.755711e+02 -4.753702e+02 ||    458.60   2.371141e-01 -1.091662e+02 -1.089291e+02 || 
     3   335.99   1.737204e-01 -4.755515e+02 -4.753778e+02 ||    450.97   2.331715e-01 -1.091906e+02 -1.089575e+02 || 
     4   276.04   1.427260e-01 -4.755253e+02 -4.753826e+02 ||    479.40   2.478694e-01 -1.092167e+02 -1.089688e+02 || 
     5   270.92   1.400757e-01 -4.754974e+02 -4.753573e+02 ||    538.20   2.782732e-01 -1.092427e+02 -1.089644e+02 || 
     6   214.75   1.110335e-01 -4.754676e+02 -4.753565e+02 ||    572.65   2.960805e-01 -1.092624e+02 -1.089663e+02 || 
     7   156.99   8.117014e-02 -4.754414e+02 -4.753603e+02 ||    617.62   3.193333e-01 -1.092692e+02 -1.089499e+02 || 
     8   138.61   7.166889e-02 -4.754189e+02 -4.753472e+02 ||    681.27   3.522421e-01 -1.092571e+02 -1.089048e+